In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-m

In [ ]:
from datasets import load_dataset
triviaqa_dataset = load_dataset("trivia_qa", "rc")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/138384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17210 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
print("entity_pages: ", triviaqa_dataset["train"][0]["entity_pages"])
print("Question: ", triviaqa_dataset["train"][0]["question"])
print("Answer: ", triviaqa_dataset["train"][0]["answer"])
print("question_source: ", triviaqa_dataset["train"][0]["question_source"])
print("question_id: ", triviaqa_dataset["train"][0]["question_id"])


entity_pages:  {'doc_source': [], 'filename': [], 'title': [], 'wiki_context': []}
Question:  Which American-born Sinclair won the Nobel Prize for Literature in 1930?
Answer:  {'aliases': ['(Harry) Sinclair Lewis', 'Harry Sinclair Lewis', 'Lewis, (Harry) Sinclair', 'Grace Hegger', 'Sinclair Lewis'], 'normalized_aliases': ['grace hegger', 'lewis harry sinclair', 'harry sinclair lewis', 'sinclair lewis'], 'matched_wiki_entity_name': '', 'normalized_matched_wiki_entity_name': '', 'normalized_value': 'sinclair lewis', 'type': 'WikipediaEntity', 'value': 'Sinclair Lewis'}
question_source:  http://www.triviacountry.com/
question_id:  tc_1


In [ ]:
triviaqa_dataset["train"].filter(lambda x: len(x["answer"]["aliases"]) != 1)

Filter:   0%|          | 0/138384 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
    num_rows: 131898
})

In [ ]:
print(triviaqa_dataset["train"])
print("entity_pages: ", triviaqa_dataset["train"][1000]["entity_pages"])
print("Question: ", triviaqa_dataset["train"][1000]["question"])
print("Answer: ", triviaqa_dataset["train"][100]["answer"])
print("question_source: ", triviaqa_dataset["train"][100]["question_source"])
print("question_id: ", triviaqa_dataset["train"][100]["question_id"])

entity_pages:  {'doc_source': ['TagMe'], 'filename': ['Vietnam.txt'], 'title': ['Vietnam'], 'wiki_context': ['Vietnam (,;  ), officially the Socialist Republic of Vietnam (SRV;  ()), is the easternmost country on the Indochina Peninsula in Southeast Asia. With an estimated 90.5 million inhabitants , it is the world\'s 14th-most-populous country, and the eighth-most-populous Asian country. Vietnam is bordered by China to the north, Laos to the northwest, Cambodia to the southwest, and Malaysia across the South China Sea to the southeast. Its capital city has been Hanoi since the reunification of North and South Vietnam in 1975.\n\nVietnam was part of Imperial China for over a millennium, from 111 BC to AD 939. An independent Vietnamese state was formed in 939, following a Vietnamese victory in the Battle of Bạch Đằng River. Successive Vietnamese royal dynasties flourished as the nation expanded geographically and politically into Southeast Asia, until the Indochina Peninsula was coloniz

In [ ]:
training_data = []
count = 0  # Counter to keep track of the number of examples added
for example in triviaqa_dataset["train"]:
    question_id = example["question_id"]
    question = example["question"]
    answer = example["answer"]["value"]

    context = example["entity_pages"]["wiki_context"]
    training_data.append({"question": question, "context": context, "answer": answer})

    count += 1
    if count >= 1000:
        break

# Print the first example for verification
print(training_data[100])



{'question': 'What is the distance between bases on a little league baseball field?', 'context': ['A baseball field, also called a ball field or a baseball diamond, is the field upon which the game of baseball is played.  The term is also used as a metonym for baseball park.\n\nSpecifications\n\nUnless otherwise noted, the specifications discussed in this section refer to those described within the Official Baseball Rules, under which Major League Baseball is played\n\nThe starting point for much of the action on the field is home plate, which is a five-sided slab of whitened rubber, 17 inches square with two of the corners removed so that one edge is 17\xa0inches long, two adjacent sides are 8½ inches and the remaining two sides are 12\xa0inches and set at an angle to make a point. Adjacent to each of the two parallel 8½-inch sides is a batter\'s box. The point of home plate where the two 12-inch sides meet at right angles is at one corner of a ninety-foot square. The other three corn

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
answer = triviaqa_dataset["train"][0]["answer"]["value"]  # Retrieve the actual answer string
question = triviaqa_dataset["train"][0]["question"]

inputs = tokenizer(question, answer, max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,return_offsets_mapping=True)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
inputs.keys()
overflow_to_sample_mapping = inputs['overflow_to_sample_mapping']
print(overflow_to_sample_mapping)


[0]


In [ ]:
answers = [triviaqa_dataset["train"][i]["answer"]["value"] for i in range(2, 6)]
questions = [triviaqa_dataset["train"][i]["question"] for i in range(2, 6)]

inputs = tokenizer(questions, answers, max_length=100,
                   truncation="only_second",
                   stride=50,
                   return_overflowing_tokens=True,
                   return_offsets_mapping=True)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")


The 4 examples gave 4 features.
Here is where each comes from: [0, 1, 2, 3].


In [ ]:
import torch

# Assuming you have a pre-trained QA model and a tokenizer
# tokenizer and model initialization
# tokenizer = ...
# model = ...

# Prepare batches
batch_size = 2
input_ids = inputs["input_ids"]
attention_masks = inputs["attention_mask"]  # Assuming the tokenizer provides attention masks
overflow_to_sample_mapping = inputs["overflow_to_sample_mapping"]

num_samples = len(overflow_to_sample_mapping)
batches = []

for i in range(0, num_samples, batch_size):
    batch_input_ids = input_ids[i:i + batch_size]
    batch_attention_masks = attention_masks[i:i + batch_size]
    batch_overflow_to_sample_mapping = overflow_to_sample_mapping[i:i + batch_size]

    # Pad inputs within each batch
    max_length = max(len(ids) for ids in batch_input_ids)
    padded_input_ids = [ids + [tokenizer.pad_token_id] * (max_length - len(ids)) for ids in batch_input_ids]
    padded_attention_masks = [masks + [0] * (max_length - len(masks)) for masks in batch_attention_masks]

    # Convert lists to tensors
    batch_input_ids_tensor = torch.tensor(padded_input_ids)
    batch_attention_masks_tensor = torch.tensor(padded_attention_masks)

    batches.append((batch_input_ids_tensor, batch_attention_masks_tensor, batch_overflow_to_sample_mapping))




In [ ]:
import torch
trained_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint)

from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForQuestionAnswering, AdamW
import torch

# Load TriviaQA dataset
triviaqa_dataset = load_dataset("trivia_qa", "rc")

# Initialize fast tokenizer and model (using BERT for this example)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Example: extract a few instances
num_examples = 4
examples = [triviaqa_dataset["train"][i] for i in range(num_examples)]
questions = [example["question"] for example in examples]

# Safely extract contexts
contexts = []
for example in examples:
    if "search_results" in example and example["search_results"]:
        if isinstance(example["search_results"], list) and len(example["search_results"]) > 0:
            context = example["search_results"][0].get("snippet", "")
        else:
            context = ""
    else:
        context = ""
    contexts.append(context)

answers = [example["answer"]["value"] for example in examples]

# Print the first example for verification
print("Question:", questions[0])
print("Context:", contexts[0])
print("Answer:", answers[0])

# Tokenize questions and contexts
inputs = tokenizer(questions, contexts, max_length=512, truncation=True, padding=True, return_tensors='pt', return_offsets_mapping=True)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
offset_mapping = inputs['offset_mapping']

# Print tokenized input for verification
print("Tokenized Input IDs:", input_ids[0])
print("Offset Mapping:", offset_mapping[0])

# Find start and end positions of answers
start_positions = []
end_positions = []

for i in range(num_examples):
    context = contexts[i]
    answer = answers[i]
    start_char = context.find(answer)
    end_char = start_char + len(answer)

    # If the answer is not found in the context, set start and end positions to 0
    if start_char == -1:
        start_positions.append(0)
        end_positions.append(0)
        continue

    # Find token positions
    sequence_ids = inputs.sequence_ids(i)
    context_start = None
    for idx, seq_id in enumerate(sequence_ids):
        if seq_id == 1:
            context_start = idx
            break

    if context_start is None:
        start_positions.append(0)
        end_positions.append(0)
        continue

    start_token_idx = None
    end_token_idx = None

    for idx, offset in enumerate(offset_mapping[i]):
        if offset[0] <= start_char and offset[1] >= start_char:
            start_token_idx = idx
        if offset[0] <= end_char and offset[1] >= end_char:
            end_token_idx = idx
            break

    if start_token_idx is None or end_token_idx is None:
        start_token_idx = context_start
        end_token_idx = context_start

    start_positions.append(start_token_idx)
    end_positions.append(end_token_idx)

start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

# Print start and end positions for verification
print("Start Positions:", start_positions)
print("End Positions:", end_positions)

# Perform forward pass
outputs = model(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

# Compute the loss
loss = outputs.loss
print(f"Loss: {loss.item()}")

# Perform backward pass and optimization
optimizer = AdamW(model.parameters(), lr=2e-5)
optimizer.zero_grad()
loss.backward()
optimizer.step()


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: Which American-born Sinclair won the Nobel Prize for Literature in 1930?
Context: 
Answer: Sinclair Lewis
Tokenized Input IDs: tensor([  101,  2029,  2137,  1011,  2141, 11881,  2180,  1996, 10501,  3396,
         2005,  3906,  1999,  4479,  1029,   102,   102])
Offset Mapping: tensor([[ 0,  0],
        [ 0,  5],
        [ 6, 14],
        [14, 15],
        [15, 19],
        [20, 28],
        [29, 32],
        [33, 36],
        [37, 42],
        [43, 48],
        [49, 52],
        [53, 63],
        [64, 66],
        [67, 71],
        [71, 72],
        [ 0,  0],
        [ 0,  0]])
Start Positions: tensor([0, 0, 0, 0])
End Positions: tensor([0, 0, 0, 0])
Loss: 2.665454864501953


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create a dataset and dataloader
dataset = TensorDataset(input_ids, attention_mask, start_positions, end_positions)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Example validation data (using a small subset for simplicity)
val_examples = [triviaqa_dataset["validation"][i] for i in range(num_examples)]
val_questions = [example["question"] for example in val_examples]
val_contexts = []
for example in val_examples:
    if "search_results" in example and example["search_results"]:
        if isinstance(example["search_results"], list) and len(example["search_results"]) > 0:
            context = example["search_results"][0].get("snippet", "")
        else:
            context = ""
    else:
        context = ""
    val_contexts.append(context)

val_answers = [example["answer"]["value"] for example in val_examples]

val_inputs = tokenizer(val_questions, val_contexts, max_length=512, truncation=True, padding=True, return_tensors='pt', return_offsets_mapping=True)
val_input_ids = val_inputs['input_ids']
val_attention_mask = val_inputs['attention_mask']
val_offset_mapping = val_inputs['offset_mapping']

val_start_positions = []
val_end_positions = []

for i in range(num_examples):
    context = val_contexts[i]
    answer = val_answers[i]
    start_char = context.find(answer)
    end_char = start_char + len(answer)

    if start_char == -1:
        val_start_positions.append(0)
        val_end_positions.append(0)
        continue

    sequence_ids = val_inputs.sequence_ids(i)
    context_start = None
    for idx, seq_id in enumerate(sequence_ids):
        if seq_id == 1:
            context_start = idx
            break

    if context_start is None:
        val_start_positions.append(0)
        val_end_positions.append(0)
        continue

    start_token_idx = None
    end_token_idx = None

    for idx, offset in enumerate(val_offset_mapping[i]):
        if offset[0] <= start_char and offset[1] >= start_char:
            start_token_idx = idx
        if offset[0] <= end_char and offset[1] >= end_char:
            end_token_idx = idx
            break

    if start_token_idx is None or end_token_idx is None:
        start_token_idx = context_start
        end_token_idx = context_start

    val_start_positions.append(start_token_idx)
    val_end_positions.append(end_token_idx)

val_start_positions = torch.tensor(val_start_positions)
val_end_positions = torch.tensor(val_end_positions)

val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_start_positions, val_end_positions)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [ ]:
from tqdm import tqdm

# Training loop
epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Training Epoch {epoch+1}"):
        batch_input_ids, batch_attention_mask, batch_start_positions, batch_end_positions = batch

        optimizer.zero_grad()

        outputs = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        start_positions=batch_start_positions,
                        end_positions=batch_end_positions)

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_train_loss:.4f}")

    # Evaluate on validation set
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Validating Epoch {epoch+1}"):
            batch_input_ids, batch_attention_mask, batch_start_positions, batch_end_positions = batch

            outputs = model(input_ids=batch_input_ids,
                            attention_mask=batch_attention_mask,
                            start_positions=batch_start_positions,
                            end_positions=batch_end_positions)

            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1} - Average Validation Loss: {avg_val_loss:.4f}")


Training Epoch 1: 100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


Epoch 1 - Average Training Loss: 2.4411


Validating Epoch 1: 100%|██████████| 1/1 [00:00<00:00,  1.43it/s]


Epoch 1 - Average Validation Loss: 2.3179


Training Epoch 2: 100%|██████████| 1/1 [00:03<00:00,  3.11s/it]


...

Epoch 2 - Average Validation Loss: 2.1951


Training Epoch 3: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


Epoch 3 - Average Training Loss: 2.1195


Validating Epoch 3: 100%|██████████| 1/1 [00:00<00:00,  2.58it/s]

Epoch 3 - Average Validation Loss: 2.0649


In [ ]:
# Example test data (using a small subset for simplicity)
test_examples = [triviaqa_dataset["test"][i] for i in range(num_examples)]
test_questions = [example["question"] for example in test_examples]
test_contexts = []
for example in test_examples:
    if "search_results" in example and example["search_results"]:
        if isinstance(example["search_results"], list) and len(example["search_results"]) > 0:
            context = example["search_results"][0].get("snippet", "")
        else:
            context = ""
    else:
        context = ""
    test_contexts.append(context)

test_answers = [example["answer"]["value"] for example in test_examples]

test_inputs = tokenizer(test_questions, test_contexts, max_length=512, truncation=True, padding=True, return_tensors='pt', return_offsets_mapping=True)
test_input_ids = test_inputs['input_ids']
test_attention_mask = test_inputs['attention_mask']
test_offset_mapping = test_inputs['offset_mapping']

test_start_positions = []
test_end_positions = []

for i in range(num_examples):
    context = test_contexts[i]
    answer = test_answers[i]
    start_char = context.find(answer)
    end_char = start_char + len(answer)

    if start_char == -1:
        test_start_positions.append(0)
        test_end_positions.append(0)
        continue

    sequence_ids = test_inputs.sequence_ids(i)
    context_start = None
    for idx, seq_id in enumerate(sequence_ids):
        if seq_id == 1:
            context_start = idx
            break

    if context_start is None:
        test_start_positions.append(0)
        test_end_positions.append(0)
        continue

    start_token_idx = None
    end_token_idx = None

    for idx, offset in enumerate(test_offset_mapping[i]):
        if offset[0] <= start_char and offset[1] >= start_char:
            start_token_idx = idx
        if offset[0] <= end_char and offset[1] >= end_char:
            end_token_idx = idx
            break

    if start_token_idx is None or end_token_idx is None:
        start_token_idx = context_start
        end_token_idx = context_start

    test_start_positions.append(start_token_idx)
    test_end_positions.append(end_token_idx)

test_start_positions = torch.tensor(test_start_positions)
test_end_positions = torch.tensor(test_end_positions)

test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_start_positions, test_end_positions)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Evaluate on test set
model.eval()
total_test_loss = 0
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating on Test Set"):
        batch_input_ids, batch_attention_mask, batch_start_positions, batch_end_positions = batch

        outputs = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        start_positions=batch_start_positions,
                        end_positions=batch_end_positions)

        loss = outputs.loss
        total_test_loss += loss.item()

avg_test_loss = total_test_loss / len(test_dataloader)
print(f"Average Test Loss: {avg_test_loss:.4f}")


Evaluating on Test Set: 100%|██████████| 1/1 [00:00<00:00,  1.28it/s]

Average Test Loss: 2.1438


In [ ]:
# Save the model and tokenizer
model.save_pretrained("path_to_save_model")
tokenizer.save_pretrained("path_to_save_tokenizer")

print("Model and tokenizer saved successfully.")


Model and tokenizer saved successfully.


In [ ]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Example question and context
question = "Which American-born Sinclair won the Nobel Prize for Literature in 1930?"
context = "Upton Sinclair won the Nobel Prize for Literature in 1930."

# Tokenize question and context
inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"]

# Let's assume we've identified the positions of "Upton Sinclair" in the tokenized context
# For example purposes, these indices are illustrative; they need to be found accurately
start_index = 1  # Example start index of "Upton"
end_index = 2    # Example end index of "Sinclair"

# Extract tokens and decode the answer
answer_tokens = input_ids[0, start_index:end_index + 1]
decoded_answer = tokenizer.decode(answer_tokens)

print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {decoded_answer}")


Question: Which American-born Sinclair won the Nobel Prize for Literature in 1930?
Context: Upton Sinclair won the Nobel Prize for Literature in 1930.
Answer: which american


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define the question and context
question = "Which American-born Sinclair won the Nobel Prize for Literature in 1930?"
context = "Upton Sinclair won the Nobel Prize for Literature in 1930."

# Tokenize the input
inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Perform the forward pass to get start and end logits
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

# Find the start and end indices with the highest scores
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

# Ensure that the end index is after the start index
if end_index < start_index:
    end_index = start_index

# Extract the answer tokens and decode them
answer_tokens = input_ids[0, start_index:end_index + 1]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: Which American-born Sinclair won the Nobel Prize for Literature in 1930?
Context: Upton Sinclair won the Nobel Prize for Literature in 1930.
Answer: american - born sinclair won the nobel


In [ ]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForQuestionAnswering, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load TriviaQA dataset
triviaqa_dataset = load_dataset("trivia_qa", "rc")

# Initialize the list to hold training data
training_data = []

# Counter to keep track of the number of examples added
count = 0

# Loop through the examples in the dataset
for example in triviaqa_dataset["train"]:
    question_id = example["question_id"]
    question = example["question"]
    answer = example["answer"]["value"]

    # Extract the context
    context = " ".join(example["entity_pages"]["wiki_context"]) if "entity_pages" in example and "wiki_context" in example["entity_pages"] else ""

    training_data.append({"question": question, "context": context, "answer": answer})

    count += 1
    if count >= 50:
        break

# Initialize tokenizer and model (using BERT for this example)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Prepare input tensors
questions = [item["question"] for item in training_data]
contexts = [item["context"] for item in training_data]
answers = [item["answer"] for item in training_data]

# Tokenize questions and contexts with handling for overflowing tokens and offsets
inputs = tokenizer(questions, contexts, max_length=512, truncation=True, padding=True,
                   return_tensors='pt', return_overflowing_tokens=True, return_offsets_mapping=True)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
overflow_to_sample_mapping = inputs["overflow_to_sample_mapping"]
offset_mapping = inputs["offset_mapping"]

# Compute start and end positions
start_positions = []
end_positions = []

for i in range(len(overflow_to_sample_mapping)):
    sample_idx = overflow_to_sample_mapping[i]
    answer = answers[sample_idx]
    context = contexts[sample_idx]

    start_char = context.find(answer)
    end_char = start_char + len(answer)

    sequence_ids = inputs.sequence_ids(i)

    # Ensure sequence_ids contain the context
    if 1 not in sequence_ids:
        start_positions.append(0)
        end_positions.append(0)
        continue

    # Find start and end token indices
    context_start = sequence_ids.index(1)
    context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

    start_token_idx = None
    end_token_idx = None
    for j, (seq_id, offset) in enumerate(zip(sequence_ids, offset_mapping[i])):
        if seq_id == 1:
            if start_token_idx is None and offset[0] >= start_char:
                start_token_idx = j
            if offset[1] >= end_char:
                end_token_idx = j
                break

    if start_token_idx is None or end_token_idx is None:
        start_positions.append(0)
        end_positions.append(0)
    else:
        start_positions.append(start_token_idx)
        end_positions.append(end_token_idx)

start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

# Create dataset and dataloader
dataset = TensorDataset(input_ids, attention_mask, start_positions, end_positions)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training loop
epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Training Epoch {epoch+1}"):
        batch_input_ids, batch_attention_mask, batch_start_positions, batch_end_positions = batch

        optimizer.zero_grad()

        outputs = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        start_positions=batch_start_positions,
                        end_positions=batch_end_positions)

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_train_loss:.4f}")

# Save the model
model.save_pretrained("bert-finetuned-triviaqa")
tokenizer.save_pretrained("bert-finetuned-triviaqa")


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1: 100%|██████████| 393/393 [1:24:14<00:00, 12.86s/it]


Epoch 1 - Average Training Loss: 1.3466


Training Epoch 2: 100%|██████████| 393/393 [1:23:25<00:00, 12.74s/it]


Epoch 2 - Average Training Loss: 0.5409


Training Epoch 3: 100%|██████████| 393/393 [1:22:51<00:00, 12.65s/it]


Epoch 3 - Average Training Loss: 0.3240


('bert-finetuned-triviaqa/tokenizer_config.json',
 'bert-finetuned-triviaqa/special_tokens_map.json',
 'bert-finetuned-triviaqa/vocab.txt',
 'bert-finetuned-triviaqa/added_tokens.json',
 'bert-finetuned-triviaqa/tokenizer.json')

In [ ]:
from google.colab import files

# Zip the saved model directory
!zip -r bert_finetuned_triviaqa.zip /content/bert-finetuned-triviaqa

# Download the zipped file
files.download("bert_finetuned_triviaqa.zip")


  adding: content/bert-finetuned-triviaqa/ (stored 0%)
  adding: content/bert-finetuned-triviaqa/tokenizer_config.json (deflated 76%)
  adding: content/bert-finetuned-triviaqa/vocab.txt (deflated 53%)
  adding: content/bert-finetuned-triviaqa/special_tokens_map.json (deflated 42%)
  adding: content/bert-finetuned-triviaqa/model.safetensors (deflated 7%)
  adding: content/bert-finetuned-triviaqa/config.json (deflated 48%)
  adding: content/bert-finetuned-triviaqa/tokenizer.json (deflated 71%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load the validation set from the TriviaQA dataset
triviaqa_dataset = load_dataset("trivia_qa", "rc")
validation_data = []

# Counter to keep track of the number of examples added
count = 0

# Loop through the examples in the dataset
for example in triviaqa_dataset["validation"]:
    question_id = example["question_id"]
    question = example["question"]
    answer = example["answer"]["value"]

    # Extract the context
    context = " ".join(example["entity_pages"]["wiki_context"]) if "entity_pages" in example and "wiki_context" in example["entity_pages"] else ""

    validation_data.append({"question": question, "context": context, "answer": answer})

    count += 1
    if count >= 100:
        break

# Load the fine-tuned model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-finetuned-triviaqa')
model = BertForQuestionAnswering.from_pretrained('bert-finetuned-triviaqa')

# Prepare input tensors for validation data
questions = [item["question"] for item in validation_data]
contexts = [item["context"] for item in validation_data]
answers = [item["answer"] for item in validation_data]

# Tokenize questions and contexts with handling for overflowing tokens and offsets
inputs = tokenizer(questions, contexts, max_length=512, truncation=True, padding=True,
                   return_tensors='pt', return_overflowing_tokens=True, return_offsets_mapping=True)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
overflow_to_sample_mapping = inputs["overflow_to_sample_mapping"]
offset_mapping = inputs["offset_mapping"]

# Compute start and end positions for validation
start_positions = []
end_positions = []

for i in range(len(overflow_to_sample_mapping)):
    sample_idx = overflow_to_sample_mapping[i]
    answer = answers[sample_idx]
    context = contexts[sample_idx]

    start_char = context.find(answer)
    end_char = start_char + len(answer)

    sequence_ids = inputs.sequence_ids(i)

    # Ensure sequence_ids contain the context
    if 1 not in sequence_ids:
        start_positions.append(0)
        end_positions.append(0)
        continue

    # Find start and end token indices
    context_start = sequence_ids.index(1)
    context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

    start_token_idx = None
    end_token_idx = None
    for j, (seq_id, offset) in enumerate(zip(sequence_ids, offset_mapping[i])):
        if seq_id == 1:
            if start_token_idx is None and offset[0] >= start_char:
                start_token_idx = j
            if offset[1] >= end_char:
                end_token_idx = j
                break

    if start_token_idx is None or end_token_idx is None:
        start_positions.append(0)
        end_positions.append(0)
    else:
        start_positions.append(start_token_idx)
        end_positions.append(end_token_idx)

start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

# Create dataset and dataloader for validation
dataset = TensorDataset(input_ids, attention_mask, start_positions, end_positions)
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

# Evaluate the model
model.eval()
total_eval_loss = 0
predictions = []

for batch in tqdm(dataloader, desc="Evaluating"):
    batch_input_ids, batch_attention_mask, batch_start_positions, batch_end_positions = batch

    with torch.no_grad():
        outputs = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        start_positions=batch_start_positions,
                        end_positions=batch_end_positions)

        loss = outputs.loss
        total_eval_loss += loss.item()

        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Find the best start and end token positions
        for i in range(batch_input_ids.size(0)):
            start_idx = torch.argmax(start_logits[i])
            end_idx = torch.argmax(end_logits[i])
            predictions.append((start_idx.item(), end_idx.item()))

# Compute the average evaluation loss
avg_eval_loss = total_eval_loss / len(dataloader)
print(f"Average Evaluation Loss: {avg_eval_loss:.4f}")

# Optionally, you can compute additional metrics such as Exact Match (EM) and F1 score
# For simplicity, we are only computing and printing the average evaluation loss here.


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 784/784 [55:14<00:00,  4.23s/it]

Average Evaluation Loss: 1.1105


In [ ]:
# Download the zipped file
files.download("bert_finetuned_triviaqa.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp bert_finetuned_triviaqa.zip /content/drive/MyDrive/


In [8]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import collections

# Load the validation set from the TriviaQA dataset
triviaqa_dataset = load_dataset("trivia_qa", "rc")
validation_data = []

# Counter to keep track of the number of examples added
count = 0

# Loop through the examples in the dataset
for example in triviaqa_dataset["validation"]:
    question_id = example["question_id"]
    question = example["question"]
    answer = example["answer"]["value"]

    # Extract the context
    context = " ".join(example["entity_pages"]["wiki_context"]) if "entity_pages" in example and "wiki_context" in example["entity_pages"] else ""

    validation_data.append({"question": question, "context": context, "answer": answer})

    count += 1
    if count >= 50:
        break

# Load the fine-tuned model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Prepare input tensors for validation data
questions = [item["question"] for item in validation_data]
contexts = [item["context"] for item in validation_data]
answers = [item["answer"] for item in validation_data]

# Tokenize questions and contexts with handling for overflowing tokens and offsets
inputs = tokenizer(questions, contexts, max_length=512, truncation=True, padding=True,
                   return_tensors='pt', return_overflowing_tokens=True, return_offsets_mapping=True)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
overflow_to_sample_mapping = inputs["overflow_to_sample_mapping"]
offset_mapping = inputs["offset_mapping"]

# Compute start and end positions for validation
start_positions = []
end_positions = []

for i in range(len(overflow_to_sample_mapping)):
    sample_idx = overflow_to_sample_mapping[i]
    answer = answers[sample_idx]
    context = contexts[sample_idx]

    start_char = context.find(answer)
    end_char = start_char + len(answer)

    sequence_ids = inputs.sequence_ids(i)

    # Ensure sequence_ids contain the context
    if 1 not in sequence_ids:
        start_positions.append(0)
        end_positions.append(0)
        continue

    # Find start and end token indices
    context_start = sequence_ids.index(1)
    context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

    start_token_idx = None
    end_token_idx = None
    for j, (seq_id, offset) in enumerate(zip(sequence_ids, offset_mapping[i])):
        if seq_id == 1:
            if start_token_idx is None and offset[0] >= start_char:
                start_token_idx = j
            if offset[1] >= end_char:
                end_token_idx = j
                break

    if start_token_idx is None or end_token_idx is None:
        start_positions.append(0)
        end_positions.append(0)
    else:
        start_positions.append(start_token_idx)
        end_positions.append(end_token_idx)

start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

# Create dataset and dataloader for validation
dataset = TensorDataset(input_ids, attention_mask, start_positions, end_positions)
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

# Evaluate the model
model.eval()
total_eval_loss = 0
predictions = []

for batch in tqdm(dataloader, desc="Evaluating"):
    batch_input_ids, batch_attention_mask, batch_start_positions, batch_end_positions = batch

    with torch.no_grad():
        outputs = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        start_positions=batch_start_positions,
                        end_positions=batch_end_positions)

        loss = outputs.loss
        total_eval_loss += loss.item()

        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Find the best start and end token positions
        for i in range(batch_input_ids.size(0)):
            start_idx = torch.argmax(start_logits[i])
            end_idx = torch.argmax(end_logits[i])
            predictions.append((start_idx.item(), end_idx.item()))

# Compute the average evaluation loss
avg_eval_loss = total_eval_loss / len(dataloader)
print(f"Average Evaluation Loss: {avg_eval_loss:.4f}")

# Compute Exact Match (EM) and F1 score
def compute_em_and_f1(predictions, answers, contexts, input_ids, tokenizer):
    exact_match = 0
    f1_total = 0

    num_predictions = min(len(predictions), len(answers))  # Ensure predictions and answers have the same length

    for i, (start_idx, end_idx) in enumerate(predictions[:num_predictions]):  # Truncate predictions if necessary
        answer = answers[i]
        context = contexts[i]

        # Decode the predicted answer
        pred_ids = input_ids[i][start_idx:end_idx+1]
        pred_answer = tokenizer.decode(pred_ids, skip_special_tokens=True)

        # Compute exact match
        if pred_answer == answer:
            exact_match += 1

        # Compute F1 score
        common_tokens = collections.Counter(answer.split()) & collections.Counter(pred_answer.split())
        num_common = sum(common_tokens.values())

        if num_common == 0:
            f1_total += 0
        else:
            precision = num_common / len(pred_answer.split())
            recall = num_common / len(answer.split())
            f1 = (2 * precision * recall) / (precision + recall)
            f1_total += f1

    exact_match = exact_match / num_predictions
    f1_score = f1_total / num_predictions

    return exact_match, f1_score


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 389/389 [22:31<00:00,  3.47s/it]

Average Evaluation Loss: 6.3224


In [9]:
# Compute EM and F1
exact_match, f1_score = compute_em_and_f1(predictions, [item["answer"] for item in validation_data],
                                           [item["context"] for item in validation_data], input_ids, tokenizer)

print(f"Exact Match (EM): {exact_match:.4f}")
print(f"F1 Score: {f1_score:.4f}")


Exact Match (EM): 0.0000
F1 Score: 0.0005


In [10]:
# Print out a sample of predictions and ground truth answers for analysis
sample_size = 5  # Number of samples to print
for i in range(sample_size):
    print(f"\nExample {i + 1}:")
    print("Question:", validation_data[i]["question"])
    print("Context:", validation_data[i]["context"])
    print("Predicted Answer:", tokenizer.decode(input_ids[i][predictions[i][0]:predictions[i][1]+1], skip_special_tokens=True))
    print("Ground Truth Answer:", validation_data[i]["answer"])



Example 1:
Question: Who was the man behind The Chipmunks?
Context: 
Predicted Answer: man behind the chipmunks?
Ground Truth Answer: David Seville

Example 2:
Question: Which Lloyd Webber musical premiered in the US on 10th December 1993?
Context: Andrew Lloyd Webber, Baron Lloyd-Webber   (born 22 March 1948) is an English composer and impresario of musical theatre. 

Several of his musicals have run for more than a decade both in the West End and on Broadway. He has composed 13 musicals, a song cycle, a set of variations, two film scores, and a Latin Requiem Mass. Several of his songs have been widely recorded and were hits outside of their parent musicals, notably "The Music of the Night" from The Phantom of the Opera, "I Don't Know How to Love Him" from Jesus Christ Superstar, "Don't Cry for Me, Argentina" and "You Must Love Me" from Evita, "Any Dream Will Do" from Joseph and the Amazing Technicolor Dreamcoat and "Memory" from Cats.

He has received a number of awards, including a